<a href="https://colab.research.google.com/github/cdario79/FaceDetection/blob/main/Face_Detection_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Face Detection per Fotocamere Digitali

## Descrizione e motivazioni della soluzione adottata

Il progetto consiste nella realizzazione di un sistema di **Face Detection** interamente costruito da zero, destinato all’integrazione in una nuova fotocamera digitale compatta.  
L’obiettivo è rilevare automaticamente i volti nelle immagini (soprattutto selfie con più persone), restituendo i bounding box corrispondenti per supportare l’ottimizzazione automatica delle impostazioni di scatto.

Il progetto presenta vincoli molto specifici:

- Non è consentito l’utilizzo di **modelli pre-addestrati**
- Non viene fornito **alcun dataset** predefinito
- Il sistema deve funzionare con **risorse computazionali limitate**
- È richiesto du usare **Scikit-learn** per l’addestramento del modello

&nbsp;

### Processo di analisi e ricerca

Partendo da questi vincoli, ho escluso fin da subito le soluzioni basate su deep learning, come **YOLO**, **MTCNN**, **RetinaFace** o **ResNet-based detectors**, in quanto si basano su modelli pre-addestrati e richiedono risorse computazionali e librerie non compatibili (es. TensorFlow o PyTorch).  
Anche **Viola-Jones**, seppur un metodo classico, è stato escluso perché in OpenCV è distribuito come modello XML pre-addestrato non compatibile con le restrizioni del progetto.

La prima fase di ricerca è partita su Google e su siti tecnici come Medium, Towards Data Science, StackOverflow e PyImageSearch, da cui ho ottenuto questi risultati:

-  🔗 [PyImageSearch – Face Detection with HOG + SVM](https://pyimagesearch.com/2015/05/04/basic-motion-detection-and-tracking-with-python-and-opencv/)  
- 🔗 [Towards Data Science – Before Deep Learning: Classical Face Detection](https://towardsdatascience.com/face-detection-methods-before-deep-learning-5c0be2d75057)

Queste ricerche mi hanno aiutato a identificare tre approcci classici utilizzabili:

1. **Haar-like features + Adaboost (Viola-Jones)** – scartato per il motivo già citato.
2. **LBP (Local Binary Pattern)** – semplice e veloce, ma meno efficace.
3. **HOG (Histogram of Oriented Gradients) + SVM** – metodo efficace e compatibile con Scikit-learn.

&nbsp;

#### Approfondimento HOG + SVM

Ho approfondito **HOG** partendo dal paper originale:

- **Dalal & Triggs (2005)**  
  🔗 [PDF](https://lear.inrialpes.fr/people/triggs/pubs/Dalal-cvpr05.pdf)

e la sua implementazione con `scikit-image`, che permette l’estrazione delle feature in modo semplice e veloce.  
Per la classificazione, ho scelto **SVM**, facilmente gestibile con `scikit-learn`, già usato in precedenti progetti ML e adatto a task binari come "face / non-face".

Inoltre, il sito PyImageSearch è stato molto utile per comprendere l’implementazione manuale di tecniche come:

- **Sliding Window**  
- **Non-Maximum Suppression (NMS)**  
- **Valutazione con TP/FP/FN**  
🔗 [https://pyimagesearch.com](https://pyimagesearch.com)

&nbsp;

### Soluzione finale adottata: HOG + SVM + Sliding Window

Alla luce di queste considerazioni, ho progettato una pipeline interamente personalizzata che soddisfa tutti i requisiti del progetto:

1. **Estrazione delle feature HOG** tramite `skimage.feature.hog`
2. **Classificatore SVM** (`sklearn.svm.SVC`) addestrato da zero su patch positive/negative
3. **Sliding Window** per scansionare l’immagine in fase di predizione
4. **Non-Maximum Suppression** per eliminare sovrapposizioni
5. **Visualizzazione finale** con codifica dei bounding box:
   - Verde = True Positive
   - Rosso = False Positive
   - Giallo = False Negative

&nbsp;

### Dataset utilizzato

Poiché il progetto non fornisce un dataset, ho costruito il dataset in autonomia:

- **Positive (volti)**: scaricati dal dataset **Labeled Faces in the Wild (LFW)**  
  🔗 [http://vis-www.cs.umass.edu/lfw/](http://vis-www.cs.umass.edu/lfw/)
- **Negative (non-volti)**: estratti da immagini generiche del dataset **Caltech 101**, evitando categorie con volti o animali  
  🔗 [http://www.vision.caltech.edu/Image_Datasets/Caltech101/](http://www.vision.caltech.edu/Image_Datasets/Caltech101/)
- Le annotazioni per le immagini di test sono state create manualmente in formato **Pascal VOC (XML)** usando [LabelImg](https://github.com/tzutalin/labelImg)

## DIPENDENZE

In [ ]:
!pip install -q scikit-learn scikit-image pillow matplotlib numpy lxml

## IMPORTAZIONE LIBRERIE

In [ ]:
# Librerie standard
import os
import shutil
import urllib.request
import zipfile
import tarfile
import random

# Librerie scientifiche
import numpy as np
import matplotlib.pyplot as plt

# Elaborazione immagini
from PIL import Image, ImageDraw

# Scikit-learn
from sklearn.datasets import fetch_lfw_people
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

# Scikit-image
from skimage.feature import hog
from skimage.color import rgb2gray
from skimage.transform import resize

## FUNZIONI UTILIZZATE

In [ ]:
def download_and_extract_negative_images(
    categories=None,
    output_dir="dataset/negative",
    image_size=(64, 64),
    num_images=500,
    seed=42
):
    # Percorsi temporanei
    zip_path = "caltech101.zip"
    temp_dir = "caltech101"
    tar_subdir = os.path.join(temp_dir, "caltech-101")
    tar_path = os.path.join(tar_subdir, "101_ObjectCategories.tar.gz")
    raw_extract_dir = "raw/negative"
    extract_dir = os.path.join(raw_extract_dir, "101_ObjectCategories")

    # Scarica lo ZIP
    os.makedirs(output_dir, exist_ok=True)
    print("📥 Downloading Caltech101 ZIP...")
    urllib.request.urlretrieve(
        "https://data.caltech.edu/records/mzrjq-6wc02/files/caltech-101.zip?download=1",
        zip_path
    )

    print("📦 Extracting ZIP...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(temp_dir)

    if not os.path.exists(tar_path):
        print("❌ File .tar.gz non trovato dopo unzip!")
        return

    print("📦 Extracting .tar.gz to:", extract_dir)
    os.makedirs(raw_extract_dir, exist_ok=True)
    with tarfile.open(tar_path, "r:gz") as tar_ref:
        tar_ref.extractall(raw_extract_dir)

    # Cleanup zip e cartella temporanea
    os.remove(zip_path)
    shutil.rmtree(temp_dir)
    print("🧹 Pulizia ZIP e cartella temporanea completata.")

    # Estrazione immagini negative
    print("🖼 Estrazione immagini negative...")
    if categories is None:
        categories = ['accordion', 'camera', 'airplanes', 'scissors', 'helicopter', 'chair', 'umbrella']

    all_paths = []
    random.seed(seed)

    for cat in categories:
        cat_path = os.path.join(extract_dir, cat)
        if not os.path.exists(cat_path):
            print(f"[!] Categoria non trovata: {cat}")
            continue
        files = [os.path.join(cat_path, f) for f in os.listdir(cat_path) if f.lower().endswith(('.jpg', '.jpeg'))]
        all_paths.extend(files)

    if not all_paths:
        print("❌ Nessuna immagine trovata.")
        return

    selected = random.sample(all_paths, min(num_images, len(all_paths)))

    count = 0
    for path in selected:
        try:
            img = Image.open(path).convert("RGB")
            img = img.resize(image_size)
            save_path = os.path.join(output_dir, f"nonface_{count}.jpg")
            if not os.path.exists(save_path):
                img.save(save_path)
                count += 1
        except Exception as e:
            print(f"[x] Errore con {path}: {e}")

    print(f"[✅] Salvate {count} immagini negative in {output_dir}")

    # Elimina la cartella raw dopo la copia
    if os.path.exists(raw_extract_dir):
        shutil.rmtree(raw_extract_dir)
        print("🧹 Cartella 'raw/negative' eliminata.")

In [ ]:
def download_and_extract_positive_images(
    output_dir="dataset/positive",
    image_size=(64, 64),
    num_images=500,
    seed=42
):
    os.makedirs(output_dir, exist_ok=True)
    random.seed(seed)

    print("📥 Caricamento del dataset LFW...")
    lfw = fetch_lfw_people(color=True, resize=1.0)
    images = lfw.images  # shape: (n_samples, height, width, 3)

    total_available = len(images)
    print(f"📊 Totale immagini disponibili: {total_available}")

    if total_available == 0:
        print("❌ Nessuna immagine disponibile nel dataset.")
        return

    indices = list(range(total_available))
    random.shuffle(indices)
    selected = indices[:min(num_images, total_available)]

    count = 0
    for i in selected:
        try:
            # Converti immagine float (0-1) a uint8 (0-255)
            img_array = (images[i] * 255).astype(np.uint8)
            img = Image.fromarray(img_array)
            img = img.resize(image_size)
            save_path = os.path.join(output_dir, f"face_{count}.jpg")
            if not os.path.exists(save_path):
                img.save(save_path)
                count += 1
        except Exception as e:
            print(f"[x] Errore con immagine {i}: {e}")

    print(f"[✅] Salvate {count} immagini di volti in {output_dir}")

In [ ]:
def extract_hog_features(img_np, size=(64, 64)):
    img_resized = resize(img_np, size, anti_aliasing=True)
    gray = rgb2gray(img_resized)
    features = hog(gray, pixels_per_cell=(8, 8), cells_per_block=(2, 2), feature_vector=True)
    return features

In [ ]:
def sliding_window(image_np, window_size=(64, 64), step=16):
    h, w, _ = image_np.shape
    for y in range(0, h - window_size[1], step):
        for x in range(0, w - window_size[0], step):
            window = image_np[y:y+window_size[1], x:x+window_size[0]]
            yield (x, y, window)

In [ ]:
def non_max_suppression(boxes, overlap_thresh=0.3):
    if len(boxes) == 0:
        return []

    boxes = np.array(boxes)
    x1 = boxes[:,0]
    y1 = boxes[:,1]
    x2 = boxes[:,0] + boxes[:,2]
    y2 = boxes[:,1] + boxes[:,3]
    scores = boxes[:,4]

    idxs = np.argsort(scores)
    pick = []

    while len(idxs) > 0:
        last = idxs[-1]
        pick.append(last)
        suppress = [last]
        for i in idxs[:-1]:
            xx1 = max(x1[last], x1[i])
            yy1 = max(y1[last], y1[i])
            xx2 = min(x2[last], x2[i])
            yy2 = min(y2[last], y2[i])

            w = max(0, xx2 - xx1)
            h = max(0, yy2 - yy1)
            overlap = float(w * h) / ((x2[i] - x1[i]) * (y2[i] - y1[i]))
            if overlap > overlap_thresh:
                suppress.append(i)

        idxs = np.delete(idxs, [np.where(idxs == s)[0][0] for s in suppress])

    return boxes[pick].astype(int)

In [ ]:
def detect_faces_pillow(image_path, clf, window_size=(64, 64), step=16, threshold=0.6):
    img = Image.open(image_path).convert("RGB")
    img_np = np.array(img)
    detections = []

    for (x, y, window) in sliding_window(img_np, window_size, step):
        if window.shape[0] != window_size[1] or window.shape[1] != window_size[0]:
            continue
        features = extract_hog_features(window)
        proba = clf.predict_proba([features])[0][1]
        if proba > threshold:
            detections.append([x, y, window_size[0], window_size[1], proba])

    return non_max_suppression(detections)

In [ ]:
def compute_iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interArea = max(0, xB - xA) * max(0, yB - yA)
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])
    iou = interArea / float(boxAArea + boxBArea - interArea + 1e-6)
    return iou

In [ ]:
def parse_annotations(xml_path):
    import xml.etree.ElementTree as ET
    tree = ET.parse(xml_path)
    root = tree.getroot()
    boxes = []
    for obj in root.findall("object"):
        box = obj.find("bndbox")
        x1 = int(box.find("xmin").text)
        y1 = int(box.find("ymin").text)
        x2 = int(box.find("xmax").text)
        y2 = int(box.find("ymax").text)
        boxes.append([x1, y1, x2, y2])
    return boxes

In [ ]:
def visualizza_risultati(image_path, annot_path, clf, iou_thresh=0.3):
    img = Image.open(image_path).convert("RGB")
    gt_boxes = parse_annotations(annot_path)
    pred_boxes = detect_faces_pillow(image_path, clf)
    draw = ImageDraw.Draw(img)
    matched_gt = set()

    for pb in pred_boxes:
        pb_box = [pb[0], pb[1], pb[0] + pb[2], pb[1] + pb[3]]
        matched = False
        for i, gt in enumerate(gt_boxes):
            iou = compute_iou(pb_box, gt)
            if iou >= iou_thresh and i not in matched_gt:
                matched_gt.add(i)
                draw.rectangle(pb_box, outline="green", width=2)  # TP
                matched = True
                break
        if not matched:
            draw.rectangle(pb_box, outline="red", width=2)  # FP

    for i, gt in enumerate(gt_boxes):
        if i not in matched_gt:
            draw.rectangle(gt, outline="yellow", width=2)  # FN

    plt.figure(figsize=(8, 8))
    plt.imshow(img)
    plt.axis("off")
    plt.title("TP=verde, FP=rosso, FN=giallo")
    plt.show()

In [ ]:
def load_dataset(pos_dir, neg_dir, size=(64, 64)):
    X, y = [], []
    for fname in os.listdir(pos_dir):
        path = os.path.join(pos_dir, fname)
        img = Image.open(path).convert("RGB")
        X.append(extract_hog_features(np.array(img), size))
        y.append(1)
    for fname in os.listdir(neg_dir):
        path = os.path.join(neg_dir, fname)
        img = Image.open(path).convert("RGB")
        X.append(extract_hog_features(np.array(img), size))
        y.append(0)
    return np.array(X), np.array(y)

In [ ]:
def evaluate_batch(test_images_dir, annotations_dir, clf, iou_thresh=0.3):
    total_TP, total_FP, total_FN = 0, 0, 0
    for fname in os.listdir(test_images_dir):
        if not fname.endswith(".jpg"): continue
        img_path = os.path.join(test_images_dir, fname)
        annot_path = os.path.join(annotations_dir, fname.replace(".jpg", ".xml"))
        if not os.path.exists(annot_path): continue

        gt = parse_annotations(annot_path)
        pred = detect_faces_pillow(img_path, clf)
        TP, FP, FN = 0, 0, 0
        matched_gt = set()

        for pb in pred:
            pb_box = [pb[0], pb[1], pb[0]+pb[2], pb[1]+pb[3]]
            matched = False
            for i, gt_box in enumerate(gt):
                if i in matched_gt: continue
                iou = compute_iou(pb_box, gt_box)
                if iou >= iou_thresh:
                    TP += 1
                    matched_gt.add(i)
                    matched = True
                    break
            if not matched: FP += 1
        FN = len(gt) - TP
        total_TP += TP
        total_FP += FP
        total_FN += FN

    precision = total_TP / (total_TP + total_FP + 1e-6)
    recall = total_TP / (total_TP + total_FN + 1e-6)
    f1 = 2 * precision * recall / (precision + recall + 1e-6)

    print("=== Valutazione finale ===")
    print(f"TP: {total_TP}, FP: {total_FP}, FN: {total_FN}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-score: {f1:.4f}")

## CREAZIONE DEL DATASET

In [ ]:
download_and_extract_negative_images(
    output_dir="dataset/negative",
    image_size=(64, 64),
    num_images=500,
    seed=123
)

📥 Downloading Caltech101 ZIP...
📦 Extracting ZIP...
📦 Extracting .tar.gz to: raw/negative/101_ObjectCategories
🧹 Pulizia ZIP e cartella temporanea completata.
🖼 Estrazione immagini negative...
[✅] Salvate 500 immagini negative in dataset/negative
🧹 Cartella 'raw/negative' eliminata.


In [ ]:
download_and_extract_positive_images(
    output_dir="dataset/positive",
    image_size=(64, 64),
    num_images=500,
    seed=123
)

📥 Caricamento del dataset LFW...
📊 Totale immagini disponibili: 13233
[✅] Salvate 500 immagini di volti in dataset/positive


## PREPARAZIONE DEL DATASET

In [ ]:
positive_dir = "/content/dataset/positive"
negative_dir = "/content/dataset/negative"

X, y = load_dataset(positive_dir, negative_dir)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## ADDESTRAMENTO DEL CLASSIFICATORE SVM

In [ ]:
clf = SVC(probability=True, kernel='rbf')
clf.fit(X_train, y_train)

print("Modello addestrato con successo!")

## TEST SU UN'IMMAGINE REALE

In [ ]:
# Esempio immagine test con annotation XML
test_image = "/content/test_images/img_001.jpg"
test_annot = "/content/test_annotations/img_001.xml"

boxes = detect_faces_pillow(test_image, clf)
print(f"Rilevati {len(boxes)} volti")

### Valutazione e risultati

Il modello è stato testato su immagini annotate manualmente. Sono state calcolate le metriche fondamentali:

- **Precision** → accuratezza dei rilevamenti positivi
- **Recall** → copertura dei volti effettivamente presenti
- **F1-score** → bilanciamento tra precision e recall

I risultati ottenuti sono stati più che soddisfacenti, considerando la semplicità dell’architettura e l’assenza di deep learning.

## VISUALIZZAZIONE CON TP / FP / FN COLORATI

In [ ]:
visualizza_risultati(test_image, test_annot, clf)

## VALUTAZIONE QUANTITATIVA SU TUTTE LE IMMAGINI DI TEST

In [ ]:
# Esegui la valutazione su tutto il test set
evaluate_batch("/content/test_images", "/content/test_annotations", clf)

### Conclusione

Questa soluzione rispetta pienamente **tutti i vincoli** del progetto e dimostra:

- Capacità di progettare una pipeline completa da zero
- Utilizzo efficace di tecniche classiche di Computer Vision
- Attenzione all’ottimizzazione per risorse limitate
- Competenze nella raccolta dati, addestramento, valutazione e documentazione

Inoltre, l’intera soluzione è **riproducibile**, **trasparente** e facilmente estendibile a sistemi embedded o real-time in ambienti con capacità limitate.